In [1]:
import numpy as np

import spglib as spg

from hilde.io import read
from hilde.harmonic_analysis import HarmonicAnalysis
from hilde.structure.convert import to_spglib_cell
from hilde.helpers.numerics import clean_matrix
from hilde.helpers.lattice_points import get_commensurate_q_points

In [2]:
mat = 'gan'
primitive = read(f'{mat}/geometry.in')
supercell = read(f'{mat}/geometry.in.supercell')

In [3]:
ha = HarmonicAnalysis(primitive, supercell, verbose=True)

get_lattice_points()
--------------------
Maximum number of iterations: 16

Supercell matrix:             
[[ 4  1  0]
 [ 2  4 -1]
 [-1 -1  2]]

lattice:                      
[[ 3.19284608  0.          0.        ]
 [-1.59642304  2.76508581  0.        ]
 [ 0.          0.          5.20451216]]

inv_lattice:                  
[[ 0.31320019  0.         -0.        ]
 [ 0.18082621  0.36165243 -0.        ]
 [ 0.          0.          0.19214097]]

.. found 25 (7) lattice points in 0.004s

get_lattice_points()
--------------------
Maximum number of iterations: 16

Supercell matrix:             
[[ 4  2 -1]
 [ 1  4 -1]
 [ 0 -1  2]]

lattice:                      
[[ 0.08769605  0.00723305  0.01537128]
 [-0.02505602  0.10126268  0.02305692]
 [-0.01252801  0.05063134  0.10759894]]

inv_lattice:                  
[[ 1.11749613e+01 -1.97376093e-09 -1.59642304e+00]
 [ 2.76508581e+00  1.10603433e+01 -2.76508581e+00]
 [ 1.07778672e-16 -5.20451216e+00  1.04090243e+01]]

.. found 25 (7) lattice points i

In [4]:
ilat = primitive.get_reciprocal_cell()

In [5]:
P = supercell.cell @ primitive.get_reciprocal_cell().T
clean_matrix(P)

array([[ 4.,  1.,  0.],
       [ 2.,  4., -1.],
       [-1., -1.,  2.]])

In [6]:
n_lp = np.round(np.linalg.det(P)).astype(int)
n_lp

25

In [7]:
n_qmesh = int(np.ceil(n_lp**(1/3)))
n_qmesh

3

In [8]:
my_mesh = np.round(ha.q_points @ supercell.cell.T).astype(int)
my_mesh, len(my_mesh)

(array([[ 0,  0,  0],
        [ 1,  1,  0],
        [ 1,  0,  1],
        [ 2,  1,  0],
        [ 1,  1,  1],
        [ 1,  2,  0],
        [ 2,  2,  0],
        [ 2,  1,  1],
        [ 1,  2,  1],
        [ 1,  3,  0],
        [ 3,  2,  0],
        [ 3,  1,  1],
        [ 3,  3, -1],
        [ 2,  2,  1],
        [ 2,  3,  0],
        [ 2,  4, -1],
        [ 3,  3,  0],
        [ 4,  2,  0],
        [ 4,  3, -1],
        [ 3,  4, -1],
        [ 4,  3,  0],
        [ 4,  4, -1],
        [ 3,  4,  0],
        [ 4,  4,  0],
        [ 4,  5, -1]]), 25)

In [9]:
n_qmesh = my_mesh.max(axis=0) - my_mesh.min(axis=0) + 1
n_qmesh

array([5, 6, 3])

In [10]:
my_mesh % n_qmesh

array([[0, 0, 0],
       [1, 1, 0],
       [1, 0, 1],
       [2, 1, 0],
       [1, 1, 1],
       [1, 2, 0],
       [2, 2, 0],
       [2, 1, 1],
       [1, 2, 1],
       [1, 3, 0],
       [3, 2, 0],
       [3, 1, 1],
       [3, 3, 2],
       [2, 2, 1],
       [2, 3, 0],
       [2, 4, 2],
       [3, 3, 0],
       [4, 2, 0],
       [4, 3, 2],
       [3, 4, 2],
       [4, 3, 0],
       [4, 4, 2],
       [3, 4, 0],
       [4, 4, 0],
       [4, 5, 2]])

In [11]:
ir_mapping, grid = spg.get_ir_reciprocal_mesh(n_qmesh, to_spglib_cell(primitive))
ir_grid = grid[np.unique(ir_mapping)]

# grid % n_qmesh
# for ii, n in enumerate(n_qmesh):
#     grid[:, ii] = grid[:, ii] % n
# grid

In [12]:
match_list = -np.ones(n_lp, dtype=int)
for i1, q1 in enumerate(my_mesh % n_qmesh):
    for i2, q2 in enumerate(grid % n_qmesh):
        if np.linalg.norm(q1 - q2) < 1e-12:
            match_list[i1] = i2

# assert len(np.unique(match_list)) == n_lp, (len(np.unique(match_list)), match_list)
len(np.unique(match_list)) == n_lp, len(np.unique(match_list)), np.unique(match_list, return_counts=True)

(True,
 25,
 (array([ 0,  6,  7, 11, 12, 13, 14, 16, 17, 18, 19, 23, 24, 31, 36, 37, 38,
         41, 42, 78, 79, 82, 83, 84, 89]),
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1])))

In [13]:
match_list, grid[8]

(array([ 0,  6, 31,  7, 36, 11, 12, 37, 41, 16, 13, 38, 78, 42, 17, 82, 18,
        14, 79, 83, 19, 84, 23, 24, 89]), array([-2,  1,  0], dtype=int32))

In [14]:
dct = {}
for nn, ii in enumerate(np.unique(ir_mapping)):
    dct[ii] = nn
dct

{0: 0,
 1: 1,
 2: 2,
 5: 3,
 6: 4,
 7: 5,
 8: 6,
 9: 7,
 10: 8,
 11: 9,
 12: 10,
 13: 11,
 14: 12,
 15: 13,
 16: 14,
 17: 15,
 30: 16,
 31: 17,
 32: 18,
 35: 19,
 36: 20,
 37: 21,
 38: 22,
 39: 23,
 40: 24,
 41: 25,
 42: 26,
 43: 27,
 44: 28,
 45: 29,
 46: 30,
 47: 31}

In [15]:
np.array([dct[ii] for ii in ir_mapping[match_list]])

array([ 0,  4, 17,  5, 20,  9, 10, 21, 25, 14, 11, 22, 31, 26, 15, 27, 15,
       12, 30, 26, 14, 25, 10,  9, 20])

In [16]:
ir_grid

array([[ 0,  0,  0],
       [ 1,  0,  0],
       [ 2,  0,  0],
       [ 0,  1,  0],
       [ 1,  1,  0],
       [ 2,  1,  0],
       [-2,  1,  0],
       [-1,  1,  0],
       [ 0,  2,  0],
       [ 1,  2,  0],
       [ 2,  2,  0],
       [-2,  2,  0],
       [-1,  2,  0],
       [ 0,  3,  0],
       [ 1,  3,  0],
       [ 2,  3,  0],
       [ 0,  0,  1],
       [ 1,  0,  1],
       [ 2,  0,  1],
       [ 0,  1,  1],
       [ 1,  1,  1],
       [ 2,  1,  1],
       [-2,  1,  1],
       [-1,  1,  1],
       [ 0,  2,  1],
       [ 1,  2,  1],
       [ 2,  2,  1],
       [-2,  2,  1],
       [-1,  2,  1],
       [ 0,  3,  1],
       [ 1,  3,  1],
       [ 2,  3,  1]], dtype=int32)